In [1]:
import pdal
import json
import pandas as pd
import geopandas as gpd
import numpy as np
import math
import alphashape
from sklearn.cluster import DBSCAN #, OPTICS
from sklearn import preprocessing


In [2]:
gdf_articulacao = gpd.read_file("zip://data/SIRGAS_SHP_quadriculamdt.zip!/SIRGAS_SHP_quadriculamdt/")

In [3]:
_ = gdf_articulacao.set_crs(epsg=31983, inplace=True)

In [4]:
#gdf_articulacao.set_index('qmdt_cod').iloc[3315-361].geometry.exterior.coords
coords = [[xy[0], xy[1]] for xy in gdf_articulacao.set_index('qmdt_cod').loc['3315-361'].geometry.exterior.coords]
xy_max = np.max(np.array(coords), axis=0) 
xy_min = np.min(np.array(coords), axis=0)

In [5]:
np.ceil(xy_max * 2) - np.ceil(xy_min * 2)

array([1077., 1166.])

In [6]:
np.floor(xy_min * 2)/2

array([ 323586., 7386800.])

In [7]:
resolution = 0.5

In [8]:
laz = [
    {
        "type":"readers.las",
        "filename":"sample-data/sao-paulo/MDS_3315-361_1000.laz"
    },
    {
        "filename":f"sample-results/sao-paulo/BHM-Z-3315-361.tiff",
        "gdaldriver":"GTiff",
        "width": 1077,
        "height": 1166,
        "origin_x": 323586,
        "origin_y": 7386800,
        "radius": f'{resolution * 2 * np.sqrt(2)}',
        "override_srs": "EPSG:31983",
        "output_type":"max",
        "resolution":resolution,
        "dimension": "Z",
        "data_type": "float32",
        "type": "writers.gdal",
        "gdalopts":"COMPRESS=ZSTD, PREDICTOR=3, BIGTIFF=YES",
        "where": "(Classification == 6)",
    },
    {
        "type":"filters.range",
        "limits":"Classification[6:6]"
    },
    ## TODO
    ## Tentar experimentar os dois tipos de clusteres
    # {
    #     "type":"filters.cluster",
    #     "min_points":100,
    #     "tolerance":0.3
    # },
    {
        "type":"filters.dbscan",
        "min_points":10,
        "eps":1.0,
        "dimensions":"X,Y,Z,Red,Green,Blue"
    },
    {
        "type":"writers.las",
        "filename":"sample-results/sao-paulo/Cluster-3315-361.laz",
        "extra_dims": "all",
        # "output_dims":"X,Y,Z,ClusterID"
    },
    {
        "type":"filters.hag_dem",
        "raster": "sample-data/sao-paulo/MDT-3315-361.tiff"
    },
    {
        "type":"filters.ferry",
        "dimensions":"HeightAboveGround => Z"
    },
    {
        "filename":f"sample-results/sao-paulo/BHM-3315-361.tiff",
        "gdaldriver":"GTiff",
        "output_type":"max",
        "resolution":"0.5",
        "width": 1077,
        "height": 1166,
        "origin_x": 323586,
        "origin_y": 7386800,
        # "nodata":"0",
        "data_type": "float32",
        "type": "writers.gdal",
        "where": "(Classification == 6)",
        "override_srs": "EPSG:31983"
    },
]

In [9]:
pipeline = pdal.Pipeline(json.dumps(laz))
# pipeline.validate()
n_points = pipeline.execute()
print(f'Pipeline selected {n_points} points')

Pipeline selected 3207051 points


In [10]:
arr = pipeline.arrays[0]
df = pd.DataFrame(arr)
# print(df.head().to_latex(index=False))
df.columns

Index(['X', 'Y', 'Z', 'Intensity', 'ReturnNumber', 'NumberOfReturns',
       'ScanDirectionFlag', 'EdgeOfFlightLine', 'Classification',
       'ScanAngleRank', 'UserData', 'PointSourceId', 'GpsTime', 'ScanChannel',
       'ClassFlags', 'Red', 'Green', 'Blue', 'Infrared', 'ClusterID',
       'HeightAboveGround'],
      dtype='object')

In [11]:
len(df.ClusterID.unique())

3306

In [12]:
df.ClusterID.value_counts()

-1       3156609
 2449        576
 2707        366
 947         314
 2933        240
          ...   
 672           8
 3069          8
 837           8
 1919          7
 2835          7
Name: ClusterID, Length: 3306, dtype: int64